In [1]:
import sys, os, csv
import numpy as np
import pandas as pd
import timeit
import pickle
import theano
import theano.tensor as T

import yaml

from theano import shared, function
from scipy import stats
from datetime import datetime as dt

from reslogit.models import Logit, ResNet, MLP
from reslogit.core import *
import reslogit.optimizers as optimizers

In [2]:
# constants
FLOATX = theano.config.floatX

# read data file from .csv
raw_data = pd.read_csv(os.getcwd() + '/data/data-20190702_2.csv')

# read configuration file
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
# keep track of time
config['timestamp'] = dt.now().strftime("%Y-%m-%d %H:%M:%S")

# defines the inputs and output
x_data = raw_data.iloc[:, 1:-1]
y_data = raw_data['mode']-1  # -1 for indexing at 0

# defines the list of explanatory variable names
config['variables'] = list(x_data.columns)

# number of observations
config['n_obs'] = raw_data.shape[0] 

# number of variables/choices
config['n_vars'] = x_data.shape[1]
config['n_choices'] = len(config['choices'])

# slicing index for train/valid split
slice = np.floor(0.7*config['n_obs']).astype(int)
config['slice'] = slice

# slices x and y datasets into train/valid
train_x_data, valid_x_data = x_data.iloc[:slice], x_data.iloc[slice:]
train_y_data, valid_y_data = y_data.iloc[:slice], y_data.iloc[slice:]

# load train/valid datasets into shared module
train_x_shared, train_y_shared = shared_dataset(train_x_data, train_y_data)
valid_x_shared, valid_y_shared = shared_dataset(valid_x_data, valid_y_data)

# number of train/valid batches
n_train_batches = train_y_data.shape[0] // config['batch_size']
n_valid_batches = valid_y_data.shape[0] // config['batch_size']
config['n_train_batches'] = n_train_batches
config['n_valid_batches'] = n_valid_batches

# Theano tensor variables
idx = T.lscalar()  # index to [mini]batch
x = T.matrix('x')
y = T.ivector('y')

In [3]:
if config['model_type'] == 'ResNet':
    # create ResNet model
    model = ResNet(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices'], 
        n_layers=config['n_layers']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        masks=model.params_mask, learning_rate=config['learning_rate']
    )
elif config['model_type'] == 'MLP':
    # create MLP model
    model = MLP(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices'], 
        n_layers=config['n_layers']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        learning_rate=config['learning_rate']
    )
elif config['model_type'] == 'MNL':
    # create MNL model
    config['n_layers'] = 0
    model = Logit(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        masks=model.params_mask, learning_rate=config['learning_rate']
    )
print(config['model_type'])

ResNet


In [4]:
batch_size = config['batch_size']

model.train_model = function(
    inputs=[idx],
    outputs=cost,
    updates=updates,
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

model.validate_model = function(
    inputs=[],
    outputs=cost,
    allow_input_downcast=True,
    givens={
        x: valid_x_shared,
        y: valid_y_shared,
    },
)

model.predict_model = function(
    inputs=[],
    outputs=model.errors(y),
    allow_input_downcast=True,
    givens={
        x: valid_x_shared,
        y: valid_y_shared,
    },
)
model.train_loss_model = function(
    inputs=[],
    outputs=model.errors(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared,
        y: train_y_shared,
    },
)


model.hessians = function(
    inputs=[idx],
    outputs=model.get_gessians(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

model.derivatives = function(
    inputs=[idx],
    outputs=model.get_derivatives(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

In [5]:
valid_freq = min(200, n_train_batches)
best_validation_ll = np.inf
start_time = timeit.default_timer()
done_looping = False
epoch = 0
step = 0

filename = '{}{}_bestmodel.pkl'.format(config['model_type'], config['n_layers'])
training_frame = pd.DataFrame(
    columns=['epoch', 'minibatch', 'batches', 'train_ll', 'valid_ll', 'valid_err', 'train_err']
)

while (epoch < config['n_epochs']) and (not done_looping):
    epoch = epoch + 1
    training_ll = 0
    for i in range(n_train_batches):
        # accumulating average score
        minibatch_ll = model.train_model(i)
        training_ll = (training_ll * i + minibatch_ll)/(i + 1)
        
        iteration = (epoch - 1) * n_train_batches + i

        if (iteration + 1) % valid_freq == 0:
            validation_ll = np.sum(model.validate_model())
            #################################
            # track and save training stats #
            #################################
            training_step = {
                'epoch': epoch, 
                'minibatch': i + 1, 
                'batches': n_train_batches, 
                'train_ll': training_ll * n_train_batches, 
                'valid_ll': validation_ll, 
                'valid_err': None,
                'train_err': None,
            }
            training_frame.loc[step] = training_step
            #################################

            # check prediction accuracy
            error = np.mean(model.predict_model())
            training_frame.loc[step, 'valid_err'] = error

            
            
            if validation_ll < best_validation_ll:
                print(('epoch {:d}, minibatch {:d}/{:d}, '
                       'validation likelihood {:.2f}').format(
                        epoch, i + 1, n_train_batches, validation_ll))

                # improve patience if loss improvement is good enough
                if validation_ll < best_validation_ll * config['improvement_threshold']:
                    config['patience'] = max(config['patience'], iteration * config['patience_increase'])

                # keep track of best validation score
                best_validation_ll = validation_ll

                best_error = np.mean(model.predict_model())
                training_frame.loc[step, 'valid_err'] = best_error
                print('validation error  {:.2%}'.format(best_error))

                # save the best model
                with open(filename, 'wb') as f:
                    pickle.dump([model, config], f)

            step = step + 1

        if epoch > 200:
            done_looping = True
            break

end_time = timeit.default_timer()
run_time = end_time - start_time

#############################
# parameter extraction step #
#############################
print('processing model statistics...')
with open(filename, 'rb') as f:
    model, config = pickle.load(f)

# format model beta params and ASCs
model_stat = {}
if config['model_type'] in ['MNL', 'ResNet']:
    beta = model.beta.eval().round(3)
    beta_df = pd.DataFrame(beta, index=config['variables'], columns=config['choices'])
    model_stat['beta_params'] = beta_df

    asc = model.params[1].eval().round(3)
    asc_df = pd.DataFrame(asc.reshape((1,-1)), index=['ASC'], columns=config['choices'])
    model_stat['asc_params'] = asc_df
    
    # compute sandwich estimator "middle"
    B = np.mean([model.derivatives(i) for i in range(n_train_batches)], axis=0)
    B = B[0]
    model_stat['sandwich_B'] = B
    
    # compute sandwich estimator "bread"
    neg_A = np.mean([model.hessians(i) for i in range(n_train_batches)], axis=0)
    invneg_A = 1/np.diag(neg_A[0])
    model_stat['sandwich_invneg_A'] = invneg_A
    
    # compute std. err and t-stat
    h = np.mean([model.hessians(i) for i in range(n_train_batches)], axis=0)
    model_stat['beta_stderr'] = pd.DataFrame(
        np.sqrt(1/np.diag(h[0]).reshape((config['n_vars'], config['n_choices'])))/(batch_size-1), 
        index=config['variables'], 
        columns=config['choices']
    )
    model_stat['beta_t_stat'] = pd.DataFrame(
        beta / model_stat['beta_stderr'], index=config['variables'], columns=config['choices'])
    
# format ResNet residual matrix
if config['model_type'] == 'ResNet':
    model_stat['residual_matrix'] = []
    for l in range(config['n_layers']):
        # create a pandas correlation matrix table
        mat = model.resnet_layers[l].params[0].eval().round(2)
        df = pd.DataFrame(
            data=mat, index=config['choices'], columns=config['choices'])
        model_stat['residual_matrix'].append(df)

# misc: runtime and training curves
model_stat['run_time'] = str(np.round(run_time / 60., 3))+' minutes'
model_stat['training_frame'] = training_frame

# re-save model, configuration and statistics     
with open(filename, 'wb') as f:
    pickle.dump([model, config, model_stat], f)        
#############################

# print final verbose output
print(('Optimization complete with best validation likelihood of {:.2f}, '
       'and validation error of {:.2%}').format(best_validation_ll, best_error))
print(('The code run for {:d} epochs, with {:.2f} epochs/sec').format(
        epoch, 1. * epoch / run_time))
print('training complete')

epoch 1, minibatch 200/1320, validation likelihood 21941.07
validation error  41.76%
epoch 1, minibatch 400/1320, validation likelihood 20857.05
validation error  41.76%
epoch 1, minibatch 600/1320, validation likelihood 19934.73
validation error  40.51%
epoch 1, minibatch 800/1320, validation likelihood 19060.56
validation error  38.00%
epoch 1, minibatch 1000/1320, validation likelihood 18991.36
validation error  38.16%
epoch 1, minibatch 1200/1320, validation likelihood 17684.20
validation error  33.74%
epoch 2, minibatch 80/1320, validation likelihood 17282.93
validation error  31.20%
epoch 2, minibatch 280/1320, validation likelihood 17078.30
validation error  32.15%
epoch 2, minibatch 480/1320, validation likelihood 16836.37
validation error  31.21%
epoch 2, minibatch 680/1320, validation likelihood 16662.06
validation error  30.53%
epoch 2, minibatch 880/1320, validation likelihood 16604.41
validation error  29.35%
epoch 2, minibatch 1080/1320, validation likelihood 16430.09
val

C:\Users\Melvin\AppData\Local\miniconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Optimization complete with best validation likelihood of 12938.32, and validation error of 23.30%
The code run for 201 epochs, with 0.25 epochs/sec
training complete


In [6]:
# if config['model_type'] == 'ResNet': 
#     print(model_stat['residual_matrix'])
# MNL Optimization complete with best validation likelihood of 15873.71, and validation error of 27.93% $$
# MLP-2 Optimization complete with best validation likelihood of 15643.12, and validation error of 27.56% $$
# MLP-4 Optimization complete with best validation likelihood of 17406.30, and validation error of 29.74% $$
# MLP-8 Optimization complete with best validation likelihood of 17370.73, and validation error of 29.70% $$
# MLP-16 Optimization complete with best validation likelihood of 17403.81, and validation error of 29.69% $$

# ResNet-2 Optimization complete with best validation likelihood of 13443.44, and validation error of 23.87% $$
# ResNet-4 Optimization complete with best validation likelihood of 13082.63, and validation error of 23.58% $$
# ResNet-8 Optimization complete with best validation likelihood of 12894.89, and validation error of 23.33% $$
# ResNet-16 Optimization complete with best validation likelihood of 12938.32, and validation error of 23.30% $$

In [7]:
filename = 'ResNet16_bestmodel.pkl'

with open(filename, 'rb') as f:
    data = pickle.load(f)
    
model, config, model_stat = data


In [8]:
model_stat['beta_params'].stack().reset_index().iloc[[0,1,2,10,11,15,18,28,36,70,72,73,77,79,80,84,87,91,94,133,134,135,140,142,144,148]][0]

0      0.045
1     -0.448
2     -0.090
10    -1.459
11     1.162
15    -1.210
18     1.618
28    -0.586
36    -0.943
70    -0.113
72     0.817
73    -0.257
77    -0.397
79     0.303
80    -0.752
84    -0.024
87    -1.900
91    -0.187
94    -0.871
133    0.340
134   -0.705
135    0.764
140    0.276
142    0.631
144    0.851
148   -1.803
Name: 0, dtype: float64

In [9]:
model_stat['beta_stderr']

,auto,bike,transit,walk,auto+transit,other_mode,other_combi
weekend,0.005666,0.062820,0.007182,0.013898,0.036211,0.012787,0.011159
hour_8_10,0.004415,0.051062,0.005195,0.012854,0.032088,0.013137,0.008596
hour_11_13,0.006341,0.070946,0.007834,0.014844,0.038799,0.014089,0.011992
hour_14_16,0.004444,0.053321,0.005440,0.012002,0.031909,0.011885,0.008626
hour_17_19,0.003855,0.045740,0.004533,0.010695,0.026053,0.011215,0.007306
hour_20_22,0.007110,0.084556,0.008921,0.018685,0.046587,0.018607,0.013561
hour_23_1,0.012076,0.139454,0.014510,0.033464,0.077152,0.033614,0.021778
hour_2_4,0.035621,0.359446,0.038388,0.099025,0.250522,0.084403,0.054848
hour_5_7,0.006175,0.068336,0.007227,0.019585,0.048162,0.019822,0.011061
num_coord,0.000452,0.005767,0.000540,0.001356,0.003936,0.001386,0.000857


In [10]:
b = model_stat["sandwich_B"]
a = model_stat["sandwich_invneg_A"]
c = np.sqrt(a * (1/b * 1/b) * a)

rob_std_err = pd.DataFrame(
        c.reshape((config['n_vars'], config['n_choices']))/(batch_size-1), 
        index=config['variables'], 
        columns=config['choices']
    )
rob_std_err

,auto,bike,transit,walk,auto+transit,other_mode,other_combi
weekend,1.156669,7.565965,0.088680,0.097709,0.393126,0.181601,0.017154
hour_8_10,0.068015,91.887547,0.014976,0.062611,0.229624,0.046899,0.013320
hour_11_13,0.003399,15.564892,0.006344,0.049166,0.359257,0.035510,0.010275
hour_14_16,0.001853,12.137209,0.002688,0.027027,0.186261,0.020284,0.010870
hour_17_19,0.000665,1.722573,0.000959,0.018023,0.120391,0.015747,0.011888
hour_20_22,0.042434,15.009122,0.049688,0.742373,1.049267,0.492509,0.091310
hour_23_1,0.189017,225.046284,0.083367,0.958466,14.867521,0.769554,0.449750
hour_2_4,1.775040,1823.264528,7.383716,236.438258,417.196739,199.283599,6.784421
hour_5_7,0.061609,12.137953,0.071081,0.241200,0.786415,0.254598,0.087260
num_coord,0.000001,0.003935,0.000002,0.000029,0.000342,0.000020,0.000007
